# Simple statistics for the "Du bist Am Zug" project

## Project links
https://dubistamzug.net/en/

https://www.instagram.com/dubistamzugberlin/

https://www.facebook.com/dubistamzugberlin

## Getting .kml map
Current notebook is tailored to work with the 2024 version of it, some things could be different for the future versions

Open [map](https://www.google.com/maps/d/u/0/viewer?mid=1jXqAMP9-YYyS75qjMMC6zf45UsSkVIs&ll=52.530777634910116%2C13.465575394245812&z=10),
press on three vertical dots, choose "Download KML", choose "Export to KML instead of KMZ", press "OK"

In [1]:
import re
import logging
import urllib.request
import xml.etree.ElementTree as ET

from collections import Counter


STYLE_YELLOW = "#icon-1899-FFD600"
STYLE_RED = "#icon-1899-A52714"


# Google map id and url that is used to download it

#mid = "1jXqAMP9-YYyS75qjMMC6zf45UsSkVIs"  # Week 1&2
mid = "1wRl3iviQosW3gyxi463XCI5FKFUd5h4"  # Week 3
download_url = f"https://www.google.com/maps/d/kml?mid={mid}&forcekml=1"

# download the map into that file
file_name = "map.kml"
urllib.request.urlretrieve(download_url, 'map.kml')

tree = ET.parse(file_name)
root = tree.getroot()

In [2]:
def norm_text(text: str) -> str:
    """Remove all html tags and fix some common typos in the source file"""
    norm = re.sub(r"<.*?>", " ", text).strip()
    norm = re.sub("8Foto", "(Foto", norm)
    norm = re.sub(r"\s+", " ", norm)

    # if there is no normal description and it is just coordinates just return empty line
    if not re.sub(r"[\d\.\s]", "", norm):
        return ""

    return norm.strip()

def get_spotter(text: str) -> str:
    """
    Try to extract spotter from the description
    
    Examples:
        >>> get_spotter("Author Name (Foto SpotterName)   52.45022583 13.50795078")
        'SpotterName'
        >>> get_spotter("Anonym (Bild SpotterName SpotterSurname)  52.55495479 13.39554122")
        'SpotterName SpotterSurname'
        >>> get_spoter("Author Name, 1, 4 (Foto Spotter1) (Foto2 Spotter2)  52.49294, 13.3868")
        'Spotter1'
    """
    if "(" in text and ")" in text:
        tt = text
        tt = re.sub(r"[Ff]oto?:?", "", tt)
        tt = re.sub(r"[Bb]ild:?", "", tt)
        return tt.split("(")[1].split(")")[0].strip()
    elif "(" in text:
        return text.split("(")[1].split(" ")[1].strip()
    logging.warning(f"No rules to parse correctly: {text}")
    return ""

def parse_description(text: str) -> str:
    """Parse description into a dictionary with various helper fields"""
    norm = norm_text(text)
    res = {"raw": text, "norm": norm}
    if not norm:
        return res
    res["poster_by"] = norm.split("(")[0].strip()  # any text before first '('
    res["spotted_by"] = get_spotter(norm)
    return res


def placemark_to_dict(placemark: ET) -> dict:
    """Convert <Placemark> structure into the dictionary"""
    name = placemark.find('kml:name', namespace).text
    styleUrl = placemark.find('kml:styleUrl', namespace).text
    description = placemark.find('kml:description', namespace).text
    description = parse_description(description)
    coordinates = placemark.find('kml:Point/kml:coordinates', namespace).text.strip()
    coordinates = coordinates.split(",")[:2]
    return {
        "name": name,
        "style": styleUrl,
        "description": description,
        "coordinates": coordinates,
    }


# parse everything

namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

document_name = root.find("kml:Document/kml:name", namespace).text
document_name = re.sub(r'[\\/:*?"<>|]', '_', document_name)

unmarked = 0
locations = []
for placemark in root.findall('.//kml:Placemark', namespace):
    locations.append(placemark_to_dict(placemark))


In [3]:
# Ensure that all parsed yellow locations have non-empty poster_by and spotted_by fields
for l in locations:
    if l["style"] != STYLE_YELLOW:
        continue
    d = l["description"]
    if not d["norm"]:
        continue
    if not d["spotted_by"] or not d["poster_by"]:
        logging.warning(f"No poster/spotter for {l['name']}: {d}")

In [4]:
# typos search, show all descriptions that don't have Foto or Bild in the description
for l in locations:
    if l["style"] != STYLE_YELLOW:
        continue
    d = l["description"]
    if not ("(Foto " in d["norm"] or "(Bild " in d["norm"]):
        print("-", l["name"], "DESCR:", re.sub(r"\s+", "  ", d["norm"]))

- Bachstr.-ggü. 1 Mittelinsel vor Altonaer Str. DESCR: Anonym  (Fot  ftravelerty)  52.51864624  13.33963776
- Wartenberger Str.-ggü. 24 vor ggü. Gehrenseestr. Rtg. Rhinstr. DESCR: Ilia  Saraantsev  (Fotostricktdagegen)  52.55092966  13.50942449
- Mehringdamm-21 vor Blücherstr. DESCR: Eva  Mauermann  (Fot  angiie_pamela_photography)  52.4957106  13.38982103
- Marktstr.-2-3 vor Schreiberhauer Str. DESCR: Alexandra  Benkenstein  (FotoTim)  52.50303388  13.47421921
- Allee der Kosmonauten-136 Mittelinsel hinter ggü. Poelchaustr. staw. DESCR: Emiliy  Püttner  (stricktdagegen)  52.53387612  13.55257366
- Allee der Kosmonauten-27 Mittelinsel vor Rhinstr. stew. DESCR: Stefanie  Schairer  (stricktdagegen)  52.52593712  13.52055927
- Britzer Damm-111 hinter Blaschkoallee DESCR: Ebrahim  Ehrari  (angiie_pamela_photography)  52.45304871  13.43853474
- Karl-Marx-Str.-208 vor ggü. Kirchhofstr. staw. DESCR: Vivia  Wisperwind  (sabineberlin.de)  52.4716301  13.44112682
- Rudower Str.-120 vor Johannist

In [5]:
print("Locations with the same name if any")
name_counter = Counter(l["name"] for l in locations)
for name, count in name_counter.most_common():
    if count > 1:
        print("-", name, ":", count)

print()
print("Locations with the same coord if any")
coord_counter = Counter(" ".join(l["coordinates"]) for l in locations)
for coord, count in coord_counter.most_common():
    if count > 1:
        print("-", coord, ":", count)

Locations with the same name if any
- Lyonel-Feininger-Str.-1 vor Nelly-Sachs-Str. : 3
- Lindenstr.-116 hinter Brandesstr. : 3
- Landsberger Allee- hinter Raoul-Wallenberg-Str. 74 : 3
- Neuköllner Str.-306 vor ggü. Alt-Rudow : 3
- Wollankstr.-22 hinter Schulzestr. : 3
- Oraniendamm-28 vor Zehntwerderweg : 3
- Roedernallee-119 hinter Am Nordgraben stew. : 3
- Martin-Luther-Str.-105 vor Dominicusstr. 2 : 3
- Neuendorfer Str.-98 vor Schönwalder Str. 1 : 3
- Gottlieb-Dunkel-Str.-28 vor Ullsteinstr. : 3
- Lankwitzer Str.- vor Großbeerenstr. 22 : 3
- Berliner Allee-217 vor Rennbahnstr. : 3
- Königstr.- vor Kronprinzessinnenweg 260 : 3
- Alice-Salomon-Platz-3 vor Hellersdorfer Str. : 2
- Lindauer Allee-ggü. 35 Mittelinsel hinter ggü. Aroser Allee : 2
- Mariendorfer Damm ggü.-8 Mittelinsel vor Ullsteinstr. stew. : 2
- Brandenburgische Str.-87 Mittelinsel vor Berliner Str. : 2
- Kaiser-Friedrich-Str.-37A Mittelinsel vor Schillerstr. stew. : 2
- Wintersteinstr.- hinter Otto-Suhr-Allee 108 : 2
- 

In [6]:
# If you are wanting to check if there are some posters you are intereted in
# just replace names here. Could be any string. I have that only because
# searching for a name or substring in google maps don't really work well for now (imho)
POSTERS_TO_CHECK = [
    "Nosyrev",
    "Kaltauskaite",
    "Dvayaitca",
    "Holubeva",
    "Pasichnyk",
    "Saliukhina",
]
for l in locations:
    d = l["description"]
    if not d["norm"]:
        continue
    for p in POSTERS_TO_CHECK:
        if p.lower() in d["poster_by"].lower():
            print(p)
            print("  ", l["name"])
            print("  ", d["norm"])

Pasichnyk
   Berliner Allee-210 hinter Bernkasteler Str. staw.
   Anastasiia Pasichnyk (Foto Stefanie J.) 52.55745316 13.46674633
Saliukhina
   Landsberger Allee-68 A-D Mittelinsel vor Petersburger Str. staw. Halle B
   Valentina Saliukhina (Foto Tim) 52.52611179 13.44693543
Pasichnyk
   Lindenstr.-116 hinter Brandesstr.
   Anastasiia Pasichnyk (Foto sabineberlin.de) 52.49893951 13.39549828
Nosyrev
   Landsberger Allee- hinter Raoul-Wallenberg-Str. 74
   Grigory Nosyrev (Foto stricktdagegen) 52.54486465 13.56416512
Kaltauskaite
   Ringstr.- hinter Kadettenweg
   Liuba Kaltauskaite (Foto Svenja) 52.43798828 13.29550648
Dvayaitca
   Am Bhf. Grunewald- vor Auerbacher Str. 2
   Paska Dvayaitca (Foto Svenja) 52.4873694 13.26339587


# Statistics

In [7]:
print("Different dots on the map count")
print("Red:", sum(1 for l in locations if l["style"] == STYLE_RED))
print("Yellow:", sum(1 for l in locations if l["style"] == STYLE_YELLOW))

Different dots on the map count
Red: 375
Yellow: 617


In [8]:
print("Different styles of dots, sanity check:")
print(Counter(l["style"] for l in locations))

Different styles of dots, sanity check:
Counter({'#icon-1899-FFD600': 617, '#icon-1899-A52714': 375, '#icon-1899-BDBDBD': 15, '#icon-1899-FFEA00': 2})


In [9]:
# Some sanity check
print("Yellow locations without any description:")
for l in locations:
    d = l["description"]
    if l["style"] != STYLE_YELLOW:
        continue
    if not d["norm"]:
        print(l["name"])

Yellow locations without any description:
Hohenstaufenstr.-60 hinter Eisenacher Str.
Schwedenstr.- vor Osloer Str. staw.


In [10]:
print("Most common posters")
posters_by_stat = Counter(l["description"].get("poster_by", "NOT_FOUND") for l in locations if l["style"] == STYLE_YELLOW)
for poster_by, count in posters_by_stat.most_common(10):
    print(f"{count}: {poster_by}")

print()
print(len(set(posters_by_stat)), "unique posters out of", sum(1 for l in locations if l["style"] == STYLE_YELLOW), "in total")

print()
print("<how_many_times_poster_was_spotted>: <posters_that_were_spotted_that_many_times>")
for freq, posters_like_that in sorted(Counter(posters_by_stat.values()).items(), reverse=True):
    print(f"{freq}: {posters_like_that}")


Most common posters
60: Anonym
3: Johannes Pol
3: Janine Pommerenke
3: Stephane Aspe
3: Lina, 8
3: Larissa Niemeyer
2: Britta Avalon Kagels
2: Konrad Rempe
2: Miriam Schimka
2: Nina-Maria Büttner

463 unique posters out of 617 in total

<how_many_times_poster_was_spotted>: <posters_that_were_spotted_that_many_times>
60: 1
3: 5
2: 85
1: 372


In [11]:
n = 10
print(f"Top {n} spotters:")
spotted_by_stat = Counter(l["description"].get("spotted_by", "NOT_FOUND") for l in locations if l["style"] == STYLE_YELLOW)
for spotted_by, count in spotted_by_stat.most_common(n):
    print(f"{count}: {spotted_by}")

print()
print(len(set(spotted_by_stat)), "unique spotters spotted", sum(spotted_by_stat.values()), "posters")

print()
print("Spotter statistics:")
print("<number_of_spotted_posters>: <spotters_that_spotted_that_many_posters>")
for freq, posters_like_that in sorted(Counter(spotted_by_stat.values()).items(), reverse=True):
    print(f"{freq}: {posters_like_that}")

Top 10 spotters:
112: Svenja
97: Tim
82: Anke
57: angiie_pamela_photography
49: stricktdagegen
36: sabineberlin.de
23: Stefanie J.
19: grigorynosyrev
13: instateechen
11: ftravelerty

55 unique spotters spotted 617 posters

Spotter statistics:
<number_of_spotted_posters>: <spotters_that_spotted_that_many_posters>
112: 1
97: 1
82: 1
57: 1
49: 1
36: 1
23: 1
19: 1
13: 1
11: 1
9: 1
8: 2
7: 1
6: 3
5: 2
4: 4
3: 1
2: 8
1: 23


In [12]:
from pathlib import Path

import pandas as pd

df = pd.DataFrame()
df["name"] = [l["name"] for l in locations]
df["description"] = [l["description"]["norm"] for l in locations]
df["coordinates"] = [" ".join(l["coordinates"]) for l in locations]
df["style"] = [l["style"] for l in locations]
df = df.sort_values("name")
df.head()

# saving file to .csv format
df.to_csv(Path(document_name).with_suffix(".csv"), index=False)

